In [4]:
!pip install langchain langchain_community chromadb openai tiktoken langchain_openai langgraph pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.6 MB/s eta 0:00

In [5]:
import os
import getpass

# Langchain and Langsmith APIs
# Langchain: A framework to build LLM-powered applications by composing prompts, models, tools, memory, and chains.
# Langsmith: A debugging, observability, and evaluation platform for LangChain workflows. Helps inspect inputs, outputs, traces, and improve LLM app quality.

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [6]:
# OpenAI API
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [7]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Using GPT 3.5 for chat llm and text-embedding-3-large for embeddings
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
# llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


embeddings = OpenAIEmbeddings(model = 'text-embedding-3-large')

In [9]:
from langchain_community.document_loaders import PyPDFLoader

resume_filenames = ["Resume_Pavan.pdf", "Resume_Pavan_DS.pdf"]
all_docs = []
for filename in resume_filenames:
    if os.path.exists(filename):
        print(f"- Loading {filename}...")
        loader = PyPDFLoader(filename)
        docs_from_pdf = loader.load()

        # 3. Add the source filename to each page's metadata. This is important!
        for doc in docs_from_pdf:
            doc.metadata['source'] = filename

        all_docs.extend(docs_from_pdf)
    else:
        # If a file is not found, print a warning and continue.
        print(f"- WARNING: The file '{filename}' was not found. Skipping.")

print("\n---------------------------------")
print("Resume loading complete.")
print(f"Total pages loaded: {len(all_docs)}")
print(f"Total characters: {sum(len(doc.page_content) for doc in all_docs)}")

- Loading Resume_Pavan.pdf...
- Loading Resume_Pavan_DS.pdf...

---------------------------------
Resume loading complete.
Total pages loaded: 2
Total characters: 9400


In [5]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# # Initialize a recursive text splitter to divide the document into chunks of max 500 characters,
# # with a 200-character overlap between consecutive chunks. This overlap helps preserve context continuity.

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 1000,
#     chunk_overlap = 200,
#     separators=["\n\n", "\n", ".", " "],
#     add_start_index = True,
# )
# all_splits = text_splitter.split_documents(all_docs)

# print(f"Split blog post into {len(all_splits)} sub-documents")

In [10]:
# Custom Splitting on sections and then recursive splitting

import re
from langchain_core.documents import Document

def split_by_section(text):
    # Define major section headers
    pattern = r"(PROFESSIONAL SUMMARY|EXPERIENCE|PROJECTS|EDUCATION|TECHNICAL SKILLS)"
    splits = re.split(pattern, text)

    # Reconstruct pairs of (header, content)
    chunks = []
    for i in range(1, len(splits), 2):
        section = splits[i].strip()
        content = splits[i+1].strip()
        full_content = f"{section}\n{content}"
        chunks.append(Document(page_content=full_content, metadata={"section": section.lower()}))
    return chunks

section_splits = []
for doc in all_docs:
    if isinstance(doc, Document):
        text = doc.page_content
    else:
        text = str(doc)  # fallback if plain string
    section_splits.extend(split_by_section(text))


from langchain_text_splitters import RecursiveCharacterTextSplitter
# Initialize a recursive text splitter to divide the document into chunks of max 1000 characters,
# with a 200-character overlap between consecutive chunks. This overlap helps preserve context continuity.

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " "],
    add_start_index=True
)

all_splits = text_splitter.split_documents(section_splits)

print(f"Split resumes into {len(all_splits)} chunks")

Split resumes into 16 chunks


In [11]:
for split in all_splits:
    print(split.page_content)
    print("---------------------------------------\n")

PROFESSIONAL SUMMARY
Machine Learning Engineer with 3+ years of experience delivering AI/ML solutions across recommendation systems, generative
modeling, and NLP. Proven ability to translate advanced research on LLMs, GANs into scalable, interpretable systems that drive
measurable business impact. Skilled at bridging experimentation and deployment in high-stakes, production-grade environments.
---------------------------------------

EXPERIENCE
PayPal India | Machine Learning Engineer Jan 2021 - Jan 2024
Python, SQL, Machine Learning, CNN, NLP, Tableau Hyderabad, India
• Boosted revenue by onboarding 50+ global merchants preventing $2M in fraud losses through ML-based fraud detection
models. Designed decision tree and anomaly detection techniques to analyze transactions and identify fraudulent patterns
• Built a Siamese CNN model with custom contrastive loss to detect document’s manipulation by comparing against true
embeddings. Achieved 94% precision, reducing manual review by 45%, pr

In [12]:
from langchain_community.vectorstores import Chroma
vector_store = Chroma.from_documents(documents=all_splits,
                                    embedding=OpenAIEmbeddings())

In [13]:
from langgraph.graph import MessagesState, StateGraph
graph_builder = StateGraph(MessagesState)

In [14]:
from langchain_core.tools import tool

# Define a tool named 'retrieve' using the @tool decorator.
# This tool will return both a formatted string and the actual documents as output.
@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=7)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    # Return both the serialized string (for LLM consumption) and the raw documents (for structured use).
    return serialized, retrieved_docs

In [15]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode


def query_or_respond(state: MessagesState):
  # Bind the 'retrieve' tool to the LLM for dynamic tool-calling.
  llm_with_tools = llm.bind_tools([retrieve])
  # Invoke the LLM with the current conversation messages (state["messages"]).
  # The LLM will decide whether to call the tool or return a direct answer.
  response = llm_with_tools.invoke(state['messages'])
  return {"messages":[response]}


# Define the tool execution node.
# ToolNode handles execution of tool calls that were requested by the LLM in the previous step.
# It automatically routes tool-calls to the correct function (e.g., `retrieve`) and returns the tool output as a ToolMessage.

tools = ToolNode([retrieve])


#	•	query_or_respond() = Decision node: LLM decides whether to use a tool or not.
#	•	ToolNode([retrieve]) = Execution node: Actually runs the tool when called.

def generate(state: MessagesState):
  # Extract the most recent tool messages in reverse order (latest first)
  recent_tool_messages = []
  for message in reversed(state["messages"]):
    if message.type == "tool":
      recent_tool_messages.append(message)
    else:
      break # Stop when we hit a non-tool message to isolate the latest tool-call output

  # Reverse the order to restore chronological flow (oldest to latest)
  tool_messages = recent_tool_messages[::-1]

  docs_content = "\n\n".join(doc.content for doc in tool_messages)

  # This is our new, much more detailed system prompt.
  system_message_content = (
    "You are a professional HR assistant. Your role is to answer questions strictly based on the candidate's resume information provided below.\n\n"
    "The context block may contain excerpts from one or more resumes. Please follow these instructions:\n"
    "1. Base your answer only on the provided '---CONTEXT---' block. You may use any retrieved context to answer, even if not explicitly labeled."
    "2. Scan all chunks in the context, especially sections like EDUCATION, EXPERIENCE, and PROJECTS, before answering."
    "3. If the source is specified (e.g., 'Source Page: ...'), use it to determine which candidate's resume the content belongs to.\n"
    "4. If the information needed is not found in the context, respond with: 'This information is not available in the provided resume.' Do not guess or fabricate answers.\n"
    "5. Keep your response concise, formal, and relevant to the query.\n\n"
    "---CONTEXT---\n"
    f"{docs_content}"
    "\n---END CONTEXT---"
)

  conversation_messages = []
  for message in state["messages"]:
    if message.type in ("human","system") or (message.type == "ai" and not message.tool_calls):
      conversation_messages.append(message)

  # Prepend the system instruction and retrieved context to the conversation
  prompt = [SystemMessage(content = system_message_content)] + conversation_messages
  response = llm.invoke(prompt)
  return {"messages":[response]}

In [16]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver


graph_builder = StateGraph(MessagesState)

# Add a node that decides whether to respond directly or make a tool call
graph_builder.add_node("query_or_respond", query_or_respond)
# Add a predefined tool execution node to handle any tool calls (e.g., retrieval)
graph_builder.add_node("tools", tools)
# Add a node that generates the final answer using retrieved content (from tool messages)
graph_builder.add_node("generate", generate)

graph_builder.set_entry_point("query_or_respond")
# If the LLM response includes a tool call, route to "tools"; otherwise, end the graph.
graph_builder.add_conditional_edges(
    "query_or_respond",     # Source node
    tools_condition,        # Function that inspects response for tool calls
    {END: END, "tools": "tools"}  # Destination mapping based on condition
)

# Define that after tool execution, go to the generate node
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)


# Initialize an in-memory checkpointer to persist chat state between steps
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}

In [17]:
input_message = "Give summary about Pavan?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Give summary about Pavan?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_6QhjntrK47dsHfWbslyjGgXk)
 Call ID: call_6QhjntrK47dsHfWbslyjGgXk
  Args:
    query: Summary about Pavan
================================= Tool Message =================================
Name: retrieve

Source: {'start_index': 0, 'section': 'professional summary'}
Content: PROFESSIONAL SUMMARY
Data Scientist with 3+ years of experience developing predictive models, A/B tests, and causal inference frameworks across fraud
detection, recommendation systems, and customer analytics. Skilled at translating complex data into actionable insights through
statistical modeling, uplift modeling, and real-time dashboards. Experienced in Python, SQL, Tableau, and PySpark on AWS-scale
systems, with a track record of cross-functional collaboration and data-driven decision making.

Source:

In [66]:
import textwrap

print("Resume Q&A Chatbot (type 'exit' to quit)")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    result = graph.invoke(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config,
    )

    final_message = result["messages"][-1]
    content = final_message.content

    # Split into lines and handle numbering cleanly
    if any(content.strip().startswith(f"{i}.") for i in range(1, 4)):
        lines = content.split("\n")
        for line in lines:
            wrapped = textwrap.fill(line, width=80,
                                    subsequent_indent=' ' * 4)
            print(wrapped)
    else:
        # Fallback: paragraph wrap
        wrapped_output = textwrap.fill(content, width=80)
        print("Assistant:\n", wrapped_output)

Resume Q&A Chatbot (type 'exit' to quit)
You: Hi
Assistant:
 Hello! How can I assist you today?
You: Tell me about Pavan
Assistant:
 Pavan is a professional with experience as both a Machine Learning Engineer and
a Data Scientist. He has worked at PayPal India, where he boosted revenue by
onboarding global merchants and preventing fraud losses through ML-based fraud
detection models. Pavan is skilled in Python, SQL, Machine Learning, NLP, and
Tableau, with a track record of delivering AI/ML solutions that drive measurable
business impact. Additionally, he holds a Master of Science in Data Science from
the University at Buffalo SUNY and a Bachelor and Master of Technology in
Electrical Engineering from the Indian Institute of Technology Madras (IITM).
You: What is his Professional Expereience
Assistant:
 Pavan's professional experience includes roles as a Machine Learning Engineer
and a Data Scientist at PayPal India. Here are some highlights of his
experience:  Machine Learning Enginee

KeyboardInterrupt: Interrupted by user

In [15]:
"""import streamlit as st

st.title("Resume Q&A Chatbot")
chat_history = st.session_state.get("chat_history", [])

user_input = st.text_input("Ask about Pavan's resume:")
if user_input:
    config = {"configurable": {"thread_id": "streamlit_thread"}}
    result = graph.invoke(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config,
    )
    assistant_message = result["messages"][-1].content
    chat_history.append(("You", user_input))
    chat_history.append(("Assistant", assistant_message))
    st.session_state.chat_history = chat_history

for role, msg in chat_history:
    st.markdown(f"**{role}:** {msg}")"""

'import streamlit as st\n\nst.title("Resume Q&A Chatbot")\nchat_history = st.session_state.get("chat_history", [])\n\nuser_input = st.text_input("Ask about Pavan\'s resume:")\nif user_input:\n    config = {"configurable": {"thread_id": "streamlit_thread"}}\n    result = graph.invoke(\n        {"messages": [{"role": "user", "content": user_input}]},\n        config=config,\n    )\n    assistant_message = result["messages"][-1].content\n    chat_history.append(("You", user_input))\n    chat_history.append(("Assistant", assistant_message))\n    st.session_state.chat_history = chat_history\n\nfor role, msg in chat_history:\n    st.markdown(f"**{role}:** {msg}")'

In [18]:
import gradio as gr

# This function takes user input and returns the chatbot's string response.
# Gradio automatically handles the UI and history display.
def resume_chatbot(user_input, history):
    # Use a unique thread_id to keep conversations separate
    config = {"configurable": {"thread_id": "gradio_thread"}}

    # Run your LangGraph agent
    result = graph.invoke(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config,
    )

    # Get the final message content from the agent's response
    assistant_message = result["messages"][-1].content

    # Return ONLY the assistant's message
    return assistant_message

# Launch the Gradio interface
gr.ChatInterface(fn=resume_chatbot, title="Resume Q&A Chatbot").launch()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6859cbbecc2f973a2d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
